In [6]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,HBox, Layout,VBox
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import DataStructs
from rdkit.Chem import AllChem
import os
from urllib.parse import parse_qs,urlparse, quote
import IPython.display
from IPython.core.display import HTML

In [3]:
try:
    query = os.environ.get('QUERY_STRING', '')
    parameters = parse_qs(query)
    smiles=parameters.get('smiles')[0]
    mol=Chem.MolFromSmiles(smiles)
    molfp = AllChem.GetMorganFingerprint(mol, radius=1, useCounts=1,useFeatures=0)
    #out=smiles
    out_s=smiles
    out_2d=mol
except:
    smiles="N#CCc1c(S)cc(cc1Br)Br"
    mol=Chem.MolFromSmiles(smiles)
    molfp = AllChem.GetMorganFingerprint(mol, radius=1, useCounts=1,useFeatures=0)
    out_s='SMILES string not valid -> proceeding with N#CCc1c(S)cc(cc1Br)Br'
    out_2d=mol
    

In [64]:
server = os.environ.get('SERVER_NAME', 'localhost') 
url = "http://" + server

port = os.environ.get('SERVER_PORT', '')
if port:
    url += ":" + port
path = os.environ.get('SCRIPT_NAME', '')
if path:
    url += path
else:
    url="http://localhost:8888/voila/render/Documents/Work/Work_website/molecular-vibration-explorer/molecule.ipynb"
url="/".join(url.split("/")[:-1])

smiles_in="N#CCc1c(S)cc(cc1Br)Br"
find_url=url+ "/search.ipynb?smiles="+quote(smiles_in)

index_link = widgets.HTML(
    value="<a href="+ url+ "/index.ipynb>Go back to Index</a>",
    layout=Layout(height='35px'),
)

gold_link = widgets.HTML(
    value="<a href="+ url+ "/database.ipynb?SH=0"+">Go to Gold database</a>",
    layout=Layout(height='35px'),
)

thiol_link = widgets.HTML(
    value="<a href="+ url+ "/database.ipynb?SH=1"+">Go to Thiol database</a>",
    #layout=Layout(height='10px'),
)

Find_text = widgets.Text(
    placeholder='e.g. N#CCc1c(S)cc(cc1Br)Br',
    disabled=False, 
    layout=Layout(width='90%',height='36px')
)

Find_button = widgets.HTML(
    value=f"<a href="+ find_url+ "><font size=5>Go</a>",
    layout=Layout(width='45px',height='30px',border='solid',margin='0 0 0 0'),
)


def Find_callback(wdgt):
    find_url=url+ "/search.ipynb?smiles="+quote(wdgt.value)
    Find_button.value=f"<a href="+ find_url+ "><font size=5>Go</a>"

Find_text.on_submit(Find_callback)


Find_panel=VBox([widgets.Label(value='Enter SMILES code and press $\mathbf{Enter}$:'),
                 Find_text, Find_button],
               layout=Layout(width='40%')
               )
Link_panel=VBox([index_link,gold_link,thiol_link],layout=Layout(width='90%'))
display(HBox([Link_panel,Find_panel]))


In [65]:
display(Find_button.value)

'<a href=http://localhost:8888/voila/render/Documents/Work/Work_website/molecular-vibration-explorer/search.ipynb?smiles=abcd><font size=5>Go</a>'

### Search molecules

In [4]:
# Similarity to other molecules
def read_smiles_data(fname):
    SAMsmiles=[]
    scode=[]
    with open(fname) as inpfile:
        line = inpfile.readline()
        line = inpfile.readline()
        while line:
            scode.append(line.split()[0])
            SAMsmiles.append(line.split()[1])
            line = inpfile.readline()
    return SAMsmiles,scode

def show_similar(smfile):
    SAMsmiles,scode=read_smiles_data(smfile)
    ms=[]
    rad=1
    for s in SAMsmiles:
        if Chem.MolFromSmiles(s)==None:
            ms.append(None)
            continue
        m=Chem.MolFromSmiles(Chem.MolToSmiles(Chem.MolFromSmiles(s)))
        ms.append(m) 
    fps=[]
    for x in ms:
        if x==None:
            fps.append(None)
            continue
        fps.append(AllChem.GetMorganFingerprint(x, radius=rad, useCounts=1,useFeatures=0))

    #molfp = AllChem.GetMorganFingerprint(mol, radius=rad, useCounts=1,useFeatures=0)
    sim=np.zeros((len(fps)))
    for j, fps2 in enumerate(fps):
            if fps2==None:
                sim[j]=0
                continue
            sim[j]=DataStructs.DiceSimilarity(molfp,fps2)

    mmol=np.argsort(sim)
    simil=[]
    for mm in mmol:
        simil.append(DataStructs.DiceSimilarity(molfp,fps[mm]))

    
    return scode,mmol,simil,SAMsmiles

In [5]:
from IPython.core.display import HTML
import pandas as pd

def path_to_image_html(path):
    return '<img src="'+ path + '" width="200" >'

def show_table(page,SAM,SH,fname,mmol,simil,allsmiles):
    nummols=10
    nmols=len(mmol)
    minmol=max(0,nmols-(page)*nummols)
    maxmol=nmols-(page-1)*nummols
    molnums=np.arange(nmols-maxmol+1,nmols-minmol+1)
    pd.options.display.float_format = '{:.2f}'.format

    bestmols=reversed(mmol[minmol:maxmol])
    images = []
    mollinks=[]
    modelinks=[]
    msmiles=[]
    codes=[]
    for m in pd.Series(bestmols,dtype='int'):
        molcode=(fname[m]).split("-",1)[1]
        codes.append(molcode)
        msmiles.append(Chem.MolToSmiles(Chem.MolFromSmiles(allsmiles[m])))
        if SAM:
            images.append("data_SAM/{}.png".format(molcode))
        elif SH:
            images.append("data_SH/{}.png".format(molcode))
        else:
            images.append("data_SAu/{}.png".format(molcode))
        molurl="<a href="+ url+ "/molecule.ipynb?Molcode="+molcode+"&SH={}".format(SH)+">Go to molecule page</a>"
        modeurl="<a href="+url+"/modes.ipynb" +"?Molcode="+molcode+"&SH={}".format(SH)+">Check normal modes</a>"
        mollinks.append(molurl)
        modelinks.append(modeurl)

    if SAM:
        df = pd.DataFrame({
        'Index' : pd.Series(molnums,dtype='int'),
        'Code': pd.Series(codes),
        'Molecule': pd.Series(images),
        'Similarity' : pd.Series(reversed(simil[minmol:maxmol]),dtype='f4'),
        'SMILES': pd.Series(msmiles),
        })
    
    else:
        df = pd.DataFrame({
        'Index' : pd.Series(molnums,dtype='int'),
        'Molecule': pd.Series(images),
        'Similarity' : pd.Series(reversed(simil[minmol:maxmol]),dtype='f4'),
        'Link to molecule page': pd.Series(mollinks),
        'Link to normal mode page': pd.Series(modelinks),
        'SMILES': pd.Series(msmiles),
        })

    
    s = df.style.format(formatter=dict(Molecule=path_to_image_html,Similarity="{:.2f}"))
    cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
    }
    index_names = {
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
    }
    headers = {
        'selector': 'th:not(.index_name)',
        'props': 'background-color: grey; color: white;'
    }
    s.set_table_styles([cell_hover, index_names, headers])

    s.set_properties(**{'background-color': 'white',
                           'color': 'black',
                           'border-color': 'black',
                        'border-style':'solid',
                        'border-width':'1.5px',
                       })
    display(HTML(s.hide_index().render()))

In [8]:
nummols=10

fname1,mmol1,simil1,allsmiles1=show_similar(smfile="data_SAu/frequencies.txt") 
pages1=np.arange(1,round(len(mmol1)/nummols+1))

page1=widgets.Dropdown(options=pages1,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)

out1=widgets.interactive_output(show_table,{'page':page1,'SAM':fixed(0),'SH':fixed(0),
                                           'fname':fixed(fname1),'mmol':fixed(mmol1),
                                            'simil':fixed(simil1),'allsmiles':fixed(allsmiles1)})


fname2,mmol2,simil2,allsmiles2=show_similar(smfile="data_SH/frequencies_SH.txt")
pages2=np.arange(1,round(len(mmol2)/nummols+1))

page2=widgets.Dropdown(options=pages2,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)

out2=widgets.interactive_output(show_table,{'page':page2,'SAM':fixed(0),'SH':fixed(1),
                                           'fname':fixed(fname2),'mmol':fixed(mmol2),
                                            'simil':fixed(simil2),'allsmiles':fixed(allsmiles2)})


fname3,mmol3,simil3,allsmiles3=show_similar(smfile="data_SAM/smiles_SAM_materials.txt",                                       )
pages3=np.arange(1,round(len(mmol3)/nummols+1))

page3=widgets.Dropdown(options=pages3,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)
out3=widgets.interactive_output(show_table,{'page':page3,'SAM':fixed(1),'SH':fixed(0),
                                            'fname':fixed(fname3),'mmol':fixed(mmol3),
                                            'simil':fixed(simil3),'allsmiles':fixed(allsmiles3)})

similar  = [
VBox([page1,out1]),
VBox([page2,out2]),
VBox([page3,out3])
                ]
mol_smiles=widgets.Output()
with mol_smiles:
    display(out_s)

mol_2d=widgets.Output()
with mol_2d:
    display(out_2d)

controls = widgets.Accordion(children=similar,selected_index=None)
controls.set_title(0, 'Similar molecules in the Gold database')
controls.set_title(1, 'Similar molecules in the Thiol database')
controls.set_title(2, 'Similar self-assembly materials')

display(VBox([mol_smiles,mol_2d,controls]))